In [1]:
!wget -O arch.pgn.zst https://database.lichess.org/standard/lichess_db_standard_rated_2015-09.pgn.zst

--2024-10-09 16:59:17--  https://database.lichess.org/standard/lichess_db_standard_rated_2015-09.pgn.zst
Resolving database.lichess.org (database.lichess.org)... 141.95.66.62, 2001:41d0:700:5e3e::
Connecting to database.lichess.org (database.lichess.org)|141.95.66.62|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 547459020 (522M) [application/octet-stream]
Saving to: ‘arch.pgn.zst’

arch.pgn.zst        100%[===================>] 522.10M  16.1MB/s    in 34s     

2024-10-09 16:59:52 (15.4 MB/s) - ‘arch.pgn.zst’ saved [547459020/547459020]



In [2]:
!apt-get install zstd

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  zstd
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 603 kB of archives.
After this operation, 1,695 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 zstd amd64 1.4.8+dfsg-3build1 [603 kB]
Fetched 603 kB in 1s (481 kB/s)
Selecting previously unselected package zstd.
(Reading database ... 123621 files and directories currently installed.)
Preparing to unpack .../zstd_1.4.8+dfsg-3build1_amd64.deb ...
Unpacking zstd (1.4.8+dfsg-3build1) ...
Setting up zstd (1.4.8+dfsg-3build1) ...
Processing triggers for man-db (2.10.2-1) ...


In [3]:
!unzstd arch.pgn.zst

arch.pgn.zst        : 2903022236 bytes 


In [4]:
!wget https://github.com/Skripkon/chess-engine/raw/main/engines/torch/auxiliary_func.py
!wget https://github.com/Skripkon/chess-engine/raw/main/engines/torch/dataset.py
!wget https://github.com/Skripkon/chess-engine/raw/main/engines/torch/model.py



--2024-10-09 17:00:27--  https://github.com/Skripkon/chess-engine/raw/main/engines/torch/auxiliary_func.py
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Skripkon/chess-engine/main/engines/torch/auxiliary_func.py [following]
--2024-10-09 17:00:27--  https://raw.githubusercontent.com/Skripkon/chess-engine/main/engines/torch/auxiliary_func.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1414 (1.4K) [text/plain]
Saving to: ‘auxiliary_func.py’

auxiliary_func.py   100%[===================>]   1.38K  --.-KB/s    in 0s      

2024-10-09 17:00:27 (17.7 MB/s) - ‘auxiliary_func.py’ saved [1414/1414]


In [5]:
!pip install chess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.4/156.4 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.0-py3-none-any.whl size=148469 sha256=a684b49ea40d04ad223907ba29f2cbb3f90da7eea34f3e7a34aec974af841127
  Stored in directory: /root/.cache/pip/wheels/46/0b/07/0a760d1eb7bd47ad727299125255ca757eb468db2d0f41db57
Successfully built chess


In [6]:
import os
import numpy as np # type: ignore
import time
import torch
import torch.nn as nn # type: ignore
import torch.optim as optim # type: ignore
from torch.utils.data import DataLoader # type: ignore
from chess import pgn # type: ignore
from tqdm import tqdm # type: ignore


In [7]:
# def load_pgn(file_path):
#     games = []
#     with open(file_path, 'r') as pgn_file:
#         while True:
#             game = pgn.read_game(pgn_file)
#             if game is None:
#                 break
#             games.append(game)
#     return games

# files = [file for file in os.listdir("Lichess Elite Database") if file.endswith(".pgn")]
# LIMIT_OF_FILES = min(len(files), 28)
# games = []
# i = 1
# for file in tqdm(files):
#     games.extend(load_pgn(f"Lichess Elite Database/{file}"))
#     if i >= LIMIT_OF_FILES:
#         break
#     i += 1
with open("arch.pgn", 'r') as pgn_file:
    games = []
    count = 0
    start = time.time()
    while True:
        # start = time.time()
        game = pgn.read_game(pgn_file)
        count += 1
        print(f"\rCount games: {count} Time taken to read: {time.time() - start}s", end = '', flush=True)
        if game is None or count > 10000:
            break
        games.append(game)

Count games: 10001 Time taken to read: 0.0020973682403564453s

In [8]:
games = games[:-1]
print(f"GAMES PARSED: {len(games)}")

GAMES PARSED: 9999


In [9]:
from auxiliary_func import create_input_for_nn, encode_moves

X, y = create_input_for_nn(games)

print(f"NUMBER OF SAMPLES: {len(y)}")

y, move_to_int = encode_moves(y)
num_classes = len(move_to_int)

X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)

100%|██████████| 9999/9999 [02:45<00:00, 60.52it/s]


NUMBER OF SAMPLES: 654057


In [11]:
from dataset import ChessDataset
from model import ChessModel

# Create Dataset and DataLoader
dataset = ChessDataset(X, y)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# Model Initialization
model = ChessModel(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

Using device: cpu


In [ ]:
# num_epochs = 50
# for epoch in range(num_epochs):
#     start_time = time.time()
#     model.train()
#     running_loss = 0.0
#     for inputs, labels in tqdm(dataloader):
#         inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
#         optimizer.zero_grad()

#         outputs = model(inputs)  # Raw logits

#         # Compute loss
#         loss = criterion(outputs, labels)
#         loss.backward()

#         # Gradient clipping
#         torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

#         optimizer.step()
#         running_loss += loss.item()
#     end_time = time.time()
#     epoch_time = end_time - start_time
#     minutes: int = int(epoch_time // 60)
#     seconds: int = int(epoch_time) - minutes * 60
#     print(f'Epoch {epoch + 1 + 50}/{num_epochs + 1 + 50}, Loss: {running_loss / len(dataloader):.4f}, Time: {minutes}m{seconds}s')


num_epochs = 50
start_epoch = 0  # Can be changed if resuming training from a specific epoch
losses = []  # To store losses for plotting

for epoch in range(start_epoch, start_epoch + num_epochs):
    start_time = time.time()
    model.train()
    running_loss = 0.0

    # Track correct predictions for accuracy calculation
    correct = 0
    total = 0

    for inputs, labels in tqdm(dataloader, desc=f'Epoch {epoch + 1}/{start_epoch + num_epochs}'):
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()

        outputs = model(inputs)  # Raw logits

        # Compute loss
        loss = criterion(outputs, labels)
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        running_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Calculate epoch loss and accuracy
    epoch_loss = running_loss / len(dataloader)
    epoch_accuracy = 100 * correct / total
    losses.append(epoch_loss)  # Store loss for plotting

    # Measure epoch time
    end_time = time.time()
    epoch_time = end_time - start_time
    minutes = int(epoch_time // 60)
    seconds = int(epoch_time % 60)

    # Print epoch statistics
    print(f'Epoch {epoch + 1}/{start_epoch + num_epochs}, '
          f'Loss: {epoch_loss:.4f}, '
          f'Accuracy: {epoch_accuracy:.2f}%, '
          f'Time: {minutes}m{seconds}s')

    # Optionally update the learning rate scheduler
    # scheduler.step() if using a scheduler


Epoch 1/50:  41%|████      | 4145/10220 [08:52<15:06,  6.70it/s]

In [ ]:
import matplotlib.pyplot as plt

plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Epochs')
plt.show()